<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Creating an NLP Data Loader 


Estimated time needed: **60** minutes


As an AI engineer working on a cutting-edge language translation project, you are tasked with bridging the communication gap between speakers of different languages. Translating languages is no small feat, especially given the intricacies, nuances, and cultural contexts embedded within them. Central to the success of this endeavor is the data - large corpora of bilingual sentences that serve as the bedrock of your models.

![Sample Image](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Screenshot%202023-10-24%20at%209.54.36%E2%80%AFAM.png)
In PyTorch, the data loader plays an indispensable role in managing this vast amount of data. For natural language processing (NLP) tasks like yours, data often comes in variable lengths due to differing sentence structures and lengths across languages. The data loader efficiently batches these variable-length sequences, ensuring that your models are trained on diverse examples in every iteration. This batching is crucial for harnessing the power of parallel computation on GPUs, thus expediting the training process.

Furthermore, the data loader aids in shuffling the data set, which is vital for preventing models from memorizing the sequence of training data and promoting better generalization. Especially for NLP tasks, where data might be ordered or clustered by topics, shuffling ensures that the model remains robust and doesn't develop biases based on the order of input.

Lastly, in the world of NLP, preprocessing steps such as tokenization, padding, and numericalization are paramount. The data loader in PyTorch provides hooks that allow us to seamlessly integrate these preprocessing steps, ensuring that the raw textual data is transformed into a format that's amenable for deep learning models.
In PyTorch, the data loader plays an indispensable role in managing this vast amount of data.

In this lab, you will cover the whole process of loading and collating text data using PyTorch.



# __Table of Contents__

<ol>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#Data-set">Data set</a>
        <ol>
            <li><a href="#Data-loader">Data loader</a></li>
        </ol>
    </li>
    <li><a href="#Custom-data-set-and-data-loader-in-PyTorch">Custom data set and data loader in PyTorch</a>
        <ol>
            <li><a href="#Creating-tensors-for-custom-data-set">Creating tensors for custom data set</a></li>
            <li><a href="#Custom-collate-function">Custom collate function</a></li>
        </ol>
    </li>
    <li><a href="#Exercise">Exercise</a>
    </li>
    <li><a href="#[Optional]-Data-loader-for-German-English-translation-task">[Optional] Data loader for German-English translation task</a>
         <ol>
            <li><a href="#Translation-data-set">Translation data set</a></li>
            <li><a href="#Tokenizer-setup">Tokenizer setup</a></li>
            <li><a href="#Special-symbols">Special symbols</a></li>
            <li><a href="#Tokens-to-indices-transformation-(Vocab)">Tokens to indices transformation (Vocab)</a></li>
        </ol>
    </li>
    <li><a href="#Processing-data-in-batches">Processing data in batches</a></li>
</ol>


# Setup
## Installing required libraries


In [19]:
!pip install nltk
!pip install transformers==4.42.1
!pip install sentencepiece
!pip install spacy
!pip install numpy==1.26.0
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install torch==2.2.2 torchtext==0.17.2
!pip install torchdata==0.7.1
!pip install portalocker
!pip install numpy pandas 
!pip install numpy scikit-learn

  Using cached numpy-2.3.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.1-cp312-cp312-manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.42.1 requires numpy<2.0,>=1.17, but you have numpy 2.3.1 which is incompatible.
  Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the

You can check the installed version of each package to make sure you have set the right environment.


## Importing required libraries


In [20]:
import torchtext
print(torchtext.__version__)

0.17.2+cpu


In [21]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

## **Data set**

A data set in **PyTorch** is an object that represents a collection of data samples. Each data sample typically consists of one or more input features and their corresponding target labels. You can also use your data set to transform your data as needed.

## **Data loader**

A data loader in **PyTorch** is responsible for efficiently loading and batching data from a data set. It abstracts away the process of iterating over a data set, shuffling, and dividing it into batches for training. In NLP applications, the data loader is used to process and transform your text data, rather than just the data set.

Data loaders have several key parameters, including the data set to load from, batch size (determining how many samples per batch), shuffle (whether to shuffle the data for each epoch), and more. Data loaders also provide an iterator interface, making it easy to iterate over batches of data during training.

Now, you may ask, '**What is an iterator?**'

An iterator is an object that can be looped over. It contains elements that can be iterated through and typically includes two methods, `__iter__()` and `__next__()`. When there are no more elements to iterate over, it raises a **`StopIteration`** exception.

Iterators are commonly used to traverse large data sets without loading all elements into memory simultaneously, making the process more memory-efficient. In PyTorch, not all data sets are iterators, but all data loaders are.

In PyTorch, the data loader processes data in batches, loading and processing one batch at a time into memory efficiently. The batch size, which you specify when creating the data loader, determines how many samples are processed together in each batch. The data loader's purpose is to convert input data and labels into batches of tensors with the same shape for deep learning models to interpret.

Finally, a data loader can be used for tasks such as tokenizing, sequencing, converting your samples to the same size, and transforming your data into tensors that your model can understand.

--- 



## **Custom data set and data loader in PyTorch**
In this code snippet, you will see how to create a custom data set and use the DataLoader class in PyTorch. The data set consists of a list of random sentences, and the objective is to create batches of sentences for further processing, such as training a neural network model.

You will begin by defining a custom data set called CustomDataset. This data set inherits from the `torch.utils.data.Dataset` class and is initialized with a list of sentences. The data set comprises two essential methods:

- __init__(self, sentences): Initializes the data set with a list of sentences.
- __getitem__(self, idx): Retrieves an item (in this case, a sentence) at a specific index, idx.

Next, you create an instance of your custom data set (custom_dataset) by passing in the list of sentences. Additionally, you can specify a batch size (batch_size), which determines how many sentences will be grouped together in each batch during data loading.

You will then create a DataLoader (dataloader) by providing your custom data set and batch size to the torch.utils.data.DataLoader class. Furthermore, you set shuffle=True, indicating that the sentences will be randomly shuffled before being divided into batches. This shuffling is particularly useful for training deep learning models, as it prevents the model from learning patterns based on the order of the data.

Finally, you iterate through the DataLoader to demonstrate how data is loaded in batches. In this code, you will see that batch size is set to 2, meaning that each batch will contain two sentences. The DataLoader efficiently manages the loading of data in batches, making it suitable for training deep learning models.

During iteration, the sentences in each batch are printed to illustrate how the DataLoader groups and presents the data. This code snippet provides a fundamental example of how to set up a custom data set and data loader in PyTorch, which is a common practice in deep learning workflows.


In [22]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Define a custom dataset
class CustomDataset(Dataset): # We are creating a class for the customDataset
    def __init__(self, sentences): # the function that will be activated right away
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

# Create an instance of your custom dataset
custom_dataset = CustomDataset(sentences)

# Define batch size
batch_size = 2

# Create a DataLoader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the DataLoader
for batch in dataloader:
    print(batch)

['Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.', "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."]
["Fame's a fickle friend, Harry.", 'It is our choices, Harry, that show what we truly are, far more than our abilities.']
['Soon we must all face the choice between what is right and what is easy.', 'You are awesome!']


As shown above, the data is organized into batches of 2 sentences each. It's important to note that deep learning models can only comprehend numerical data, and words are meaningless to them. Therefore, your next step is to convert these sentences into tensors. Let's see how to do this.


In [23]:
# custom_dataset = CustomDataset(sentences)
# custom_dataset.__len__()

### Creating tensors for custom data set

In this code example, you will see the creation of a custom data set for natural language processing (NLP) tasks using PyTorch. The data set consists of a list of sentences, and your goal is to preprocess these sentences, tokenize them, and convert them into tensors of token indices for use in NLP models. Let's break down the code step by step.

The sentences and the CustomDataset class are used in the same way as in the previous code snippet. The changes made to the CustomDataset class are as follows:

- __init__: The constructor takes a list of sentences, a tokenizer function, and a vocabulary (vocab) as input.
- __len__: This method returns the total number of samples in the data set.
- __getitem__: This method is responsible for processing a single sample. It tokenizes the sentence using the provided tokenizer and then converts the tokens into tensor indices using the vocabulary.

You can define a tokenizer using the `get_tokenizer` function with the `basic_english` option. Tokenization is the process of splitting a text into individual tokens or words. Next, you build a vocabulary from the sentences. You use the `build_vocab_from_iterator` function to construct the vocabulary from the tokenized sentences.

You can create an instance of your custom data set, passing in the sentences, tokenizer, and vocabulary. Finally, you print the length of the custom data set and sample items from the data set for illustration.


In [24]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx]) # Split sentence into words
        # Convert tokens to tensor indices using vocab
        tensor_indices = [self.vocab[token] for token in tokens] # Convert words to numbers
        return torch.tensor(tensor_indices)

# Tokenizer
tokenizer = get_tokenizer("basic_english") # Creates word splitter

# Build vocabulary
vocab = build_vocab_from_iterator(map(tokenizer, sentences)) # Creates word->number mapping

# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Custom Dataset Length:", len(custom_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

Custom Dataset Length: 6
Sample Items:
Item 1: tensor([11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
        43, 61,  9, 44,  0, 14,  9, 33,  1])
Item 2: tensor([35,  6, 16,  3, 38, 40,  0,  8,  1])
Item 3: tensor([12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
        21,  1])
Item 4: tensor([54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1])
Item 5: tensor([66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
         2, 12, 64, 17, 26, 65,  1])
Item 6: tensor([19,  4, 25, 20])


Please go ahead and uncomment the following code to apply the data loader and observe the results:


In [25]:

# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

# Define batch size
batch_size = 2

# Create a data loader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the data loader
for batch in dataloader:
    print(batch)

RuntimeError: stack expects each tensor to be equal size, but got [27] at entry 0 and [16] at entry 1

You will encounter an error when attempting to create batches for the tensors. This error arises because the tensor batches have unequal lengths. The data loader is using the default `collate_function`, which requires tensors to have equal lengths. You can define your own `collate_function` and pass the data into it to establish your own rules. Typically, to address the issue of unequal tensor lengths, you employ data padding. This will be demonstrated in the following section.


### Custom collate function

A collate function is employed in the context of data loading and batching in machine learning, particularly when dealing with variable-length data, such as sequences (e.g., text, time series, sequences of events). Its primary purpose is to prepare and format individual data samples (examples) into batches that can be efficiently processed by machine learning models.

You will begin by defining a custom collate function named `collate_fn`. This function plays a crucial role when handling sequences of varying lengths, such as sentences in NLP. Its purpose is to pad sequences within a batch to have equal lengths, which is a common preprocessing step in NLP tasks.

`pad_sequence`: This function is a part of PyTorch and is utilized to pad sequences in a batch, ensuring uniform length. It takes a batch of sequences as input and pads them to match the length of the longest sequence. The `padding_value=0` argument specifies the value to use for padding.


In [ ]:
# Create a custom collate function
def collate_fn(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_batch

In the above cell, when padding the sequences, you set `batch_first=True`. When `batch_first=True`, output will be in [batch_size x seq_len] shape, otherwise it will be in [seq_len x batch_size] shape. Some models accept input with [batch_size x seq_len] shape while some other models need the input to be of [seq_len x batch_size] shape. Keep in mind that this parameter takes care of putting the input in the desired shape. 

Let's see how it actually affects the shape of curated batches. First, you create a data loader from a collate function with `batch_first=True`:


In [26]:
# Create a data loader with the custom collate function with batch_first=True,
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Iterate through the data loader
for batch in dataloader: 
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)
       

NameError: name 'collate_fn' is not defined

Looking into the result, you can see that the first dimension is the batch. For example, first batch is the first sentence: "['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']". 

Now, you can try `batch_first=False` which is the DEFAULT value:


In [27]:
# Create a custom collate function
def collate_fn_bfFALSE(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, padding_value=0)
    return padded_batch

Now, you look into the curated data:


In [28]:


# Create a data loader with the custom collate function with batch_first=True,
dataloader_bfFALSE = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn_bfFALSE)

# Iterate through the data loader
for seq in dataloader_bfFALSE:
    for row in seq:
        #print(row)
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)

['if', 'fame']
['you', "'"]
['want', 's']
['to', 'a']
['know', 'fickle']
['what', 'friend']
['a', ',']
['man', 'harry']
["'", '.']
['s', ',']
['like', ',']
[',', ',']
['take', ',']
['a', ',']
['good', ',']
['look', ',']
['at', ',']
['how', ',']
['he', ',']
['treats', ',']
['his', ',']
['inferiors', ',']
[',', ',']
['not', ',']
['his', ',']
['equals', ',']
['.', ',']
['it', 'soon']
['is', 'we']
['our', 'must']
['choices', 'all']
[',', 'face']
['harry', 'the']
[',', 'choice']
['that', 'between']
['show', 'what']
['what', 'is']
['we', 'right']
['truly', 'and']
['are', 'what']
[',', 'is']
['far', 'easy']
['more', '.']
['than', ',']
['our', ',']
['abilities', ',']
['.', ',']
['youth', 'you']
['can', 'are']
['not', 'awesome']
['know', '!']
['how', ',']
['age', ',']
['thinks', ',']
['and', ',']
['feels', ',']
['.', ',']
['but', ',']
['old', ',']
['men', ',']
['are', ',']
['guilty', ',']
['if', ',']
['they', ',']
['forget', ',']
['what', ',']
['it', ',']
['was', ',']
['to', ',']
['be', ',']
['

It can be seen that the first dimension is now the sequence instead of batch, which means sentences will break so that each row includes a token from each sequence. For example the first row, (['if', 'fame']), includes the first tokens of all the sequences in that batch. You need to be aware of this standard to avoid any confusion when working with recurrent neural networks (RNNs) and transformers.


You will see that each batch has a fixed size for all the sequences within the batch.

You also have the option to utilize the collate function for tasks such as tokenization, converting tokenized indices, and transforming the result into a tensor. It's important to note that the original data set remains untouched by these transformations.


In [30]:
# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [31]:
custom_dataset=CustomDataset(sentences)

You have the raw text:


In [32]:
custom_dataset[0]

"If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."

You create the new ```collate_fn```


In [33]:
def collate_fn(batch):
    # Tokenize each sample in the batch using the specified tokenizer
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convert tokens to vocabulary indices and create a tensor for each sample
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))

    # Pad sequences within the batch to have equal lengths using pad_sequence
    # batch_first=True ensures that the tensors have shape (batch_size, max_sequence_length)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    
    # Return the padded batch
    return padded_batch

Create a data loader with the custom collate function.


In [34]:
# Create a data loader for the custom dataset
dataloader = DataLoader(
    dataset=custom_dataset,   # Custom PyTorch Dataset containing your data
    batch_size=batch_size,     # Number of samples in each mini-batch
    shuffle=True,              # Shuffle the data at the beginning of each epoch
    collate_fn=collate_fn      # Custom collate function for processing batches
)

You will see that the result is a tensor of the same shape for each sample in the batch.


In [35]:
for batch in dataloader:
    print(batch)
    print("shape of sample",len(batch))

tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
shape of sample 2
tensor([[66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1],
        [35,  6, 16,  3, 38, 40,  0,  8,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0]])
shape of sample 2
tensor([[54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0],
        [11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1]])
shape of sample 2


As a result, batches of tensors with equal lengths have been successfully created.


# Congratulations! You have completed the lab.


## Authors


[Joseph Santarcangelo](https://www.linkedin.com/in/joseph-s-50398b136/) has a Ph.D. in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

[Roodra Kanwar](https://www.linkedin.com/in/roodrakanwar/) is completing his MS in CS specializing in big data from Simon Fraser University. He has previous experience working with machine learning and as a data engineer.

[Fateme Akbari](https://www.linkedin.com/in/fatemeakbari/) is a PhD candidate in Information Systems at McMaster University with demonstrated research experience in Machine Learning and NLP.



© Copyright IBM Corporation. All rights reserved.


```{## Change Log}
```


```{|Date (YYYY-MM-DD)|Version|Changed By|Change Description|}
```
```{|-|-|-|-|}
```
```{|2023-10-24|0.1|Roodra|Created Lab Template|}
```
